In [214]:
import pandas as pd
import numpy as np 
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import io
import re
from scipy.interpolate import interp1d
from datetime import datetime
from datetime import timezone
from math import sqrt 

In [299]:
capnormal=pd.read_csv("HA_capNormalVol.csv")
swaps_data=pd.read_csv("HA_swaps.csv")
swaption=pd.read_csv("HA_swaptionNormalVol.csv")

In [300]:
def schedule(start, end, freq=6):            
    def advance(dt, months):
        try:
            return date(dt.year, dt.month + months, dt.day)
        except:
            return date(dt.year + 1, dt.month - months, dt.day)
    dates = [start]
    while dates[-1] <= end:
        dates.append(advance(dates[-1], 6))
    return dates[:-1]

In [301]:
#Exercise 1
#creating schedule of dates
today_date = date(2007,1,15)
maturity = date(2022,1,19)
start_date = date(2007,1,19)

In [302]:
#Exercise 1
swaps = swaps_data.rename(columns={'Unnamed: 0': 'Date'}) #changing name of columns
swaps['Date'] = pd.to_datetime(swaps['Date']) #setting the Date columns to a datetime columns
swaps.set_index('Date', inplace=True) #setting the column as the index
swaps

,Deposit 6M,FRA 1X7,FRA 2X8,FRA 3X9,FRA 4X10,FRA 5X11,FRA 6X12,FRA 7X13,FRA 8X14,FRA 9X15,...,Swap 9Y,Swap 10Y,Swap 11Y,Swap 12Y,Swap 15Y,Swap 20Y,Swap 25Y,Swap 30Y,Swap 40Y,Swap 50Y
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-02,3.85700,3.92850,3.97700,4.01550,4.04550,4.07550,4.09750,4.09650,4.10100,4.10050,...,4.15200,4.17200,4.18700,4.20600,4.24450,4.28400,4.27500,4.25900,4.21600,4.17400
2007-01-03,3.86000,3.91700,3.97200,4.01450,4.04100,4.06700,4.08200,4.08700,4.09200,4.08900,...,4.15050,4.17150,4.18850,4.20450,4.25800,4.28450,4.29500,4.26350,4.22050,4.17850
2007-01-04,3.86500,3.93150,3.98650,4.02250,4.04350,4.07200,4.09150,4.09750,4.10250,4.09850,...,4.14450,4.16250,4.18150,4.19850,4.24550,4.27550,4.27800,4.25650,4.21450,4.17250
2007-01-05,3.86400,3.94150,3.99500,4.04100,4.07100,4.09500,4.12200,4.13050,4.13650,4.13250,...,4.18750,4.20550,4.22350,4.24050,4.28850,4.31750,4.32900,4.29700,4.25500,4.21500
2007-01-08,3.87400,3.94350,4.00150,4.04350,4.06950,4.09850,4.12250,4.12150,4.12750,4.12350,...,4.18600,4.20600,4.22600,4.24200,4.28850,4.32000,4.32100,4.30000,4.25900,4.22000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.16200,0.21070,NaN,0.13460,0.06100,NaN
2020-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.19350,NaN,NaN,NaN,-0.00850,0.03410,-0.00120,-0.06100,-0.15900,NaN
2019-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.30500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.57600,0.53000,NaN


In [303]:
swaps_v1=swaps.loc['2007-01-15 00:00:00'].transpose()
swaps_v1.columns=['par_rate']
swaps_v1.reset_index(inplace=True)
swaps_v1

,index,par_rate
0,Deposit 6M,3.88000
1,FRA 1X7,3.95200
2,FRA 2X8,4.02600
3,FRA 3X9,4.06590
4,FRA 4X10,4.10480
5,FRA 5X11,4.13860
6,FRA 6X12,4.15430
7,FRA 7X13,4.16060
8,FRA 8X14,4.16890
9,FRA 9X15,4.16750


In [304]:
swaps_v1[['instrument', 'maturity']] = swaps_v1['index'].str.split(' ', expand=True) #splitting the index column to have a columns with only the name of the product

In [305]:
swaps_v1["maturity"] = swaps_v1["maturity"].apply(lambda x: x + "M" if "X" in x else x) #adding a M to the FRAs maturities

In [306]:
swaps_v1['maturity_fra'] = swaps_v1['maturity'].str.split('X').str[1] #getting the maturity of the fra

In [307]:
swaps_v1.loc[swaps_v1["instrument"] == "FRA", "maturity"] = swaps_v1["maturity_fra"] #getting the new maturity

In [308]:
def extract_number(string):
    # Use a regular expression to extract the numeric value from the string
    number = re.findall(r'\d+', string)
    # If the string contains the character 'M', return the number as a float
    # (dividing the number by 12 to convert from months to years)
    if 'M' in string:
        return float(number[0]) / 12
    # Otherwise, return the number as an integer
    else:
        return int(number[0])

In [309]:
swaps_v1["year_frac"] = swaps_v1["maturity"].apply(extract_number)

In [310]:
swaps_v1.set_index('index',inplace=True)

In [311]:
forward=swaps_v1.loc['FRA 6X12', 'par_rate']

In [312]:
one_year_rate=(((1+swaps_v1.loc['Deposit 6M', 'par_rate'])**swaps_v1.loc['Deposit 6M', 'year_frac'])*(1+forward)**(1-swaps_v1.loc['Deposit 6M', 'year_frac']))-1
one_year_rate

4.015275067232105

In [313]:
yield_curve = swaps_v1[swaps_v1.index.str.contains("FRA") == False]

In [314]:
one_year_rate_frame = pd.DataFrame([[one_year_rate,'-','1Y','-',1.0]],
     index=['Swap 1Y'],
     columns=['par_rate', 'instrument','maturity','maturity_fra','year_frac'])
one_year_rate_frame

,par_rate,instrument,maturity,maturity_fra,year_frac
Swap 1Y,4.01528,-,1Y,-,1.00000


In [315]:
yield_curve=yield_curve.append(one_year_rate_frame)

In [316]:
yield_curve.set_index('year_frac',inplace=True)

In [317]:
yield_curve.loc[0.5, 'df'] = 1 / (1 + (yield_curve.loc[0.5, 'par_rate']/100) * 0.5)
yield_curve.loc[0.5, 'zero_rate'] = yield_curve.loc[0.5, 'par_rate']/100

In [318]:
yield_curve.loc[1, 'df'] = 1 / (1 + yield_curve.loc[1, 'par_rate']/100 * 1)
yield_curve.loc[1, 'zero_rate'] = yield_curve.loc[1, 'par_rate']/100

In [319]:
yield_curve=yield_curve.sort_index()

In [353]:
start_date

datetime.date(2007, 1, 19)

In [328]:
yield_curve["Tenor"]= yield_curve.index
yield_curve['maturity'] = [date(start_date.year,start_date.month+6,start_date.day)] + [date(2007 +int(n), 1, 19) for n in yield_curve.index[1:]]
yield_curve['days'] = yield_curve.maturity.apply(lambda x: (x - start_date).days)
yield_curve

,par_rate,instrument,maturity,maturity_fra,df,zero_rate,Tenor,days
year_frac,,,,,,,,
0.50000,3.88000,Deposit,2007-07-19,NaN,0.98097,0.03880,0.50000,181
1.00000,4.01528,-,2008-01-19,-,0.96140,0.04015,1.00000,365
2.00000,4.16900,Swap,2009-01-19,NaN,NaN,NaN,2.00000,731
3.00000,4.17250,Swap,2010-01-19,NaN,NaN,NaN,3.00000,1096
4.00000,4.17550,Swap,2011-01-19,NaN,NaN,NaN,4.00000,1461
5.00000,4.18450,Swap,2012-01-19,NaN,NaN,NaN,5.00000,1826
6.00000,4.20250,Swap,2013-01-19,NaN,NaN,NaN,6.00000,2192
7.00000,4.21850,Swap,2014-01-19,NaN,NaN,NaN,7.00000,2557
8.00000,4.24250,Swap,2015-01-19,NaN,NaN,NaN,8.00000,2922


In [329]:
for idx, row in yield_curve.loc[2:].iterrows():
    prev_dfs = yield_curve.loc[1:idx-1].df.sum()
    yield_curve.loc[idx, 'df'] = (1 - row.par_rate/100 * prev_dfs) / (1+row.par_rate/100)

In [330]:

for a in yield_curve.index[2:]:
    yield_curve.loc[a,'zero_rate']=((1/yield_curve.loc[a,'df'])**(1/a))-1


In [331]:
start_date = date(2007,1,19)
# Add six months and keep the same day
loan_dates=[start_date]
for i in range(30):
  resulting_date = start_date + relativedelta(months=+6)
  if resulting_date.weekday()==5:
    resulting_date=resulting_date+timedelta(days=2)
  if resulting_date.weekday()==6:
    resulting_date=resulting_date+timedelta(days=1)
  loan_dates.append(resulting_date)
  start_date=date(resulting_date.year,resulting_date.month,19)
start_date = date(2007,1,19)
loan_dates

[datetime.date(2007, 1, 19),
 datetime.date(2007, 7, 19),
 datetime.date(2008, 1, 21),
 datetime.date(2008, 7, 21),
 datetime.date(2009, 1, 19),
 datetime.date(2009, 7, 20),
 datetime.date(2010, 1, 19),
 datetime.date(2010, 7, 19),
 datetime.date(2011, 1, 19),
 datetime.date(2011, 7, 19),
 datetime.date(2012, 1, 19),
 datetime.date(2012, 7, 19),
 datetime.date(2013, 1, 21),
 datetime.date(2013, 7, 19),
 datetime.date(2014, 1, 20),
 datetime.date(2014, 7, 21),
 datetime.date(2015, 1, 19),
 datetime.date(2015, 7, 20),
 datetime.date(2016, 1, 19),
 datetime.date(2016, 7, 19),
 datetime.date(2017, 1, 19),
 datetime.date(2017, 7, 19),
 datetime.date(2018, 1, 19),
 datetime.date(2018, 7, 19),
 datetime.date(2019, 1, 21),
 datetime.date(2019, 7, 19),
 datetime.date(2020, 1, 20),
 datetime.date(2020, 7, 20),
 datetime.date(2021, 1, 19),
 datetime.date(2021, 7, 19),
 datetime.date(2022, 1, 19)]

In [332]:
nr_days=[]
for a in range(0,len(loan_dates)):
    if a ==0:
      pass
    else:
      nr_days.append((loan_dates[a]-today_date).days/365)
nr_days

[0.5068493150684932,
 1.0164383561643835,
 1.515068493150685,
 2.0136986301369864,
 2.5123287671232877,
 3.0136986301369864,
 3.5095890410958903,
 4.013698630136986,
 4.50958904109589,
 5.013698630136986,
 5.512328767123288,
 6.021917808219178,
 6.512328767123288,
 7.019178082191781,
 7.517808219178082,
 8.016438356164384,
 8.515068493150684,
 9.016438356164384,
 9.515068493150684,
 10.01917808219178,
 10.515068493150684,
 11.01917808219178,
 11.515068493150684,
 12.024657534246575,
 12.515068493150684,
 13.021917808219179,
 13.520547945205479,
 14.021917808219179,
 14.517808219178082,
 15.021917808219179]

In [333]:
y=np.array(yield_curve.zero_rate.values)
x=np.array(yield_curve.index.values)
y_interp = interp1d(x,y)
 
# Finding the interpolation
interpolated_rates=[]
for a in nr_days:
  interpolate_x=a
  interpolated_rates.append(float(y_interp(interpolate_x)))
interpolated_rates

[0.038818530831127684,
 0.0401785480772134,
 0.04096106935894772,
 0.04172244627297853,
 0.04173531257030465,
 0.04174825530203538,
 0.04176125871019032,
 0.04177540709209084,
 0.041822056504942834,
 0.04187087887849022,
 0.041968737505568496,
 0.0420683090732121,
 0.04215475890475419,
 0.04224594754610628,
 0.04238171355993787,
 0.04251715209004331,
 0.04264298443084846,
 0.04276977762419703,
 0.04290378376868447,
 0.04303887093294358,
 0.04316201576346114,
 0.043286799221674215,
 0.04339953792666865,
 0.043465482029131246,
 0.04258434181141895,
 0.04167366616741463,
 0.04077775823655632,
 0.03987692773464934,
 0.038985942374839704,
 0.03808843304905318]

In [334]:
notional=1748653.15
installemnts=np.full(30,1748653.15)
int_pay=installemnts*interpolated_rates
int_pay[len(int_pay)-1]=notional*(1+interpolated_rates[len(interpolated_rates)-1])
int_pay

array([ 67880.14621622, 70258.34465765, 71626.70296189,
 72958.08710095, 72980.5857923 , 73003.21814091,
 73025.95659154, 73050.69720412, 73132.27084685,
 73217.64424414, 73388.76504064, 73562.88117605,
 73714.05194629, 73873.50925123, 74110.91691898,
 74347.75193128, 74567.7890504 , 74789.50636735,
 75023.83663403, 75260.05722934, 75475.39482513,
 75693.5978124 , 75890.73870401, 76006.05206651,
 74465.24344921, 72872.7876157 , 71306.15539029,
 69730.91529552, 68172.89093948, 1815256.60842979])

In [335]:
#Exercise 2
#The company is Party A
swap_notional=1748653.15
start_swap=loan_dates[0]
maturity_swap=loan_dates[-1]
#Party A Receives Euribor 6m, half yearly, act/360
#Party A Pays xxxxxxxx, half yearly, act/360

In [336]:
#TESTE
yield_curve_swap = yield_curve.copy()
yield_curve_swap

,par_rate,instrument,maturity,maturity_fra,df,zero_rate,Tenor,days
year_frac,,,,,,,,
0.50000,3.88000,Deposit,2007-07-19,NaN,0.98097,0.03880,0.50000,181
1.00000,4.01528,-,2008-01-19,-,0.96140,0.04015,1.00000,365
2.00000,4.16900,Swap,2009-01-19,NaN,0.92150,0.04172,2.00000,731
3.00000,4.17250,Swap,2010-01-19,NaN,0.88453,0.04175,3.00000,1096
4.00000,4.17550,Swap,2011-01-19,NaN,0.84900,0.04177,4.00000,1461
5.00000,4.18450,Swap,2012-01-19,NaN,0.81458,0.04187,5.00000,1826
6.00000,4.20250,Swap,2013-01-19,NaN,0.78097,0.04206,6.00000,2192
7.00000,4.21850,Swap,2014-01-19,NaN,0.74856,0.04224,7.00000,2557
8.00000,4.24250,Swap,2015-01-19,NaN,0.71672,0.04251,8.00000,2922


In [337]:
def getDF(dt):
    x = (dt - start_date).days
    return np.interp(x, yield_curve_swap.days, yield_curve_swap.df)

def getFwd(start, end):
    DF0 = getDF(start)
    DF1 = getDF(end)
    n = (end - start).days / 360
    return (DF0 / DF1 - 1) / n

In [338]:
#get floating leg
pd.options.display.float_format = '{:,.5f}'.format
float_leg = pd.DataFrame({
    'notional': swap_notional,
    'start': loan_dates[:-1],
    'end': loan_dates[1:]
})

float_leg['dcf'] = float_leg.apply(lambda x: (x.end - x.start).days / 360, axis=1)
float_leg['fwd'] = float_leg.apply(lambda row: getFwd(row.start, row.end), axis=1)
float_leg.loc[0, 'fwd'] = yield_curve_swap.loc[0.5, 'par_rate']/100
float_leg['cf'] = float_leg.notional * float_leg.dcf * float_leg.fwd
float_leg['df'] = float_leg.end.apply(getDF)
float_leg['pv'] = float_leg.cf * float_leg.df


float_leg_pv = float_leg.pv.sum()
print(f"PV of float leg is: {float_leg_pv:,.2f}")
float_leg

PV of float leg is: 751,154.04


,notional,start,end,dcf,fwd,cf,df,pv
0,"1,748,653.15000",2007-01-19,2007-07-19,0.50278,0.03880,"34,112.33706",0.98097,"33,463.15191"
1,"1,748,653.15000",2007-07-19,2008-01-21,0.51667,0.03985,"36,003.43733",0.96118,"34,605.75683"
2,"1,748,653.15000",2008-01-21,2008-07-21,0.50556,0.04169,"36,852.68842",0.94134,"34,690.93149"
3,"1,748,653.15000",2008-07-21,2009-01-19,0.50556,0.04258,"37,646.07566",0.92150,"34,690.93149"
4,"1,748,653.15000",2009-01-19,2009-07-20,0.50556,0.04038,"35,698.16367",0.90307,"32,237.80296"
5,"1,748,653.15000",2009-07-20,2010-01-19,0.50833,0.04123,"36,646.54485",0.88453,"32,414.93374"
6,"1,748,653.15000",2010-01-19,2010-07-19,0.50278,0.04043,"35,542.35504",0.86691,"30,811.97511"
7,"1,748,653.15000",2010-07-19,2011-01-19,0.51111,0.04128,"36,893.77046",0.84900,"31,322.67083"
8,"1,748,653.15000",2011-01-19,2011-07-19,0.50278,0.04080,"35,868.12115",0.83193,"29,839.82821"
9,"1,748,653.15000",2011-07-19,2012-01-19,0.51111,0.04167,"37,239.12433",0.81458,"30,334.41100"


In [339]:
fixed_leg = pd.DataFrame({
    'notional': swap_notional,
    'start': loan_dates[::2][:-1],
    'end': loan_dates[::2][1:]
})

fixed_leg['dcf'] = 1
fixed_leg['df'] = fixed_leg.end.apply(getDF)
fixed_leg

,notional,start,end,dcf,df
0,"1,748,653.15000",2007-01-19,2008-01-21,1,0.96118
1,"1,748,653.15000",2008-01-21,2009-01-19,1,0.92150
2,"1,748,653.15000",2009-01-19,2010-01-19,1,0.88453
3,"1,748,653.15000",2010-01-19,2011-01-19,1,0.84900
4,"1,748,653.15000",2011-01-19,2012-01-19,1,0.81458
5,"1,748,653.15000",2012-01-19,2013-01-21,1,0.78079
6,"1,748,653.15000",2013-01-21,2014-01-20,1,0.74847
7,"1,748,653.15000",2014-01-20,2015-01-19,1,0.71672
8,"1,748,653.15000",2015-01-19,2016-01-19,1,0.68599
9,"1,748,653.15000",2016-01-19,2017-01-19,1,0.65617


In [340]:
#getting swap rate with matrix algebra
fixed_rate = float_leg_pv / (fixed_leg.notional *  fixed_leg.dcf * fixed_leg.df).sum()
print(f"Fair fixed rate is: {fixed_rate * 100:.4f}%")

Fair fixed rate is: 3.9097%


In [341]:
from scipy.optimize import root
def swapPv(rate):
    fxd = fixed_leg.copy()
    fixed_leg_pv = (fxd.notional * fxd.dcf * fxd.df * rate).sum()
    return fixed_leg_pv - float_leg_pv


res = root(swapPv, 0)
fixed_rate1 = res['x'][0]

print(f"Fixed rate Matrix Algebra:\t {fixed_rate * 100:.4f}%")
print(f"Fixed rate Optimization:\t {fixed_rate1 * 100:.4f}%")

Fixed rate Matrix Algebra:	 3.9097%
Fixed rate Optimization:	 3.9097%


In [342]:
fixed_leg['rate'] = fixed_rate
fixed_leg['cashflow'] = fixed_leg.notional * fixed_leg.dcf * fixed_leg.rate
fixed_leg['pv'] = fixed_leg.cashflow * fixed_leg.df


fixed_leg_pv = fixed_leg.pv.sum()
print(f"Present Value of Fixed Leg is: {fixed_leg_pv:,.2f}")

fixed_leg

Present Value of Fixed Leg is: 751,154.04


,notional,start,end,dcf,df,rate,cashflow,pv
0,"1,748,653.15000",2007-01-19,2008-01-21,1,0.96118,0.03910,"68,367.00997","65,712.95126"
1,"1,748,653.15000",2008-01-21,2009-01-19,1,0.92150,0.03910,"68,367.00997","63,000.33183"
2,"1,748,653.15000",2009-01-19,2010-01-19,1,0.88453,0.03910,"68,367.00997","60,472.60682"
3,"1,748,653.15000",2010-01-19,2011-01-19,1,0.84900,0.03910,"68,367.00997","58,043.33149"
4,"1,748,653.15000",2011-01-19,2012-01-19,1,0.81458,0.03910,"68,367.00997","55,690.70209"
5,"1,748,653.15000",2012-01-19,2013-01-21,1,0.78079,0.03910,"68,367.00997","53,380.21803"
6,"1,748,653.15000",2013-01-21,2014-01-20,1,0.74847,0.03910,"68,367.00997","51,170.50489"
7,"1,748,653.15000",2014-01-20,2015-01-19,1,0.71672,0.03910,"68,367.00997","48,999.84877"
8,"1,748,653.15000",2015-01-19,2016-01-19,1,0.68599,0.03910,"68,367.00997","46,899.39337"
9,"1,748,653.15000",2016-01-19,2017-01-19,1,0.65617,0.03910,"68,367.00997","44,860.22841"


In [343]:
#Question 2b)
pv01 = (fixed_leg.notional * fixed_leg.dcf * 0.0001 * fixed_leg.df).sum()

print(f"Present value of 1 bps on the fixed leg is: EUR {pv01:,.2f}")

Present value of 1 bps on the fixed leg is: EUR 1,921.26


In [344]:
#question 2c
fixed_leg_2c = pd.DataFrame({
    'notional': swap_notional,
    'start': loan_dates[::2][:-1],
    'end': loan_dates[::2][1:]
})

fixed_leg_2c['dcf'] = 1
fixed_leg_2c['df'] = fixed_leg.end.apply(getDF)
fixed_leg_2c

,notional,start,end,dcf,df
0,"1,748,653.15000",2007-01-19,2008-01-21,1,0.96118
1,"1,748,653.15000",2008-01-21,2009-01-19,1,0.92150
2,"1,748,653.15000",2009-01-19,2010-01-19,1,0.88453
3,"1,748,653.15000",2010-01-19,2011-01-19,1,0.84900
4,"1,748,653.15000",2011-01-19,2012-01-19,1,0.81458
5,"1,748,653.15000",2012-01-19,2013-01-21,1,0.78079
6,"1,748,653.15000",2013-01-21,2014-01-20,1,0.74847
7,"1,748,653.15000",2014-01-20,2015-01-19,1,0.71672
8,"1,748,653.15000",2015-01-19,2016-01-19,1,0.68599
9,"1,748,653.15000",2016-01-19,2017-01-19,1,0.65617


In [345]:
#question 2c

fixed_leg_2c['rate'] = fixed_rate+0.0005
fixed_leg_2c['cashflow'] = fixed_leg_2c.notional * fixed_leg_2c.dcf * fixed_leg_2c.rate
fixed_leg_2c['pv'] = fixed_leg_2c.cashflow * fixed_leg_2c.df


fixed_leg_pv_2c = fixed_leg_2c.pv.sum()
print(f"Present Value of Fixed Leg, with the fee, is: {fixed_leg_pv_2c:,.2f}")

Present Value of Fixed Leg, with the fee, is: 760,760.34


In [346]:
#Other way to calculate querstion 2c
pv05 = (fixed_leg.notional * fixed_leg.dcf * 0.0005 * fixed_leg.df).sum()
result_pv05 = fixed_leg_pv + pv05
print(f"The price of this swap considering that the counterparty charges 5bps is: {result_pv05:.3f}")

The price of this swap considering that the counterparty charges 5bps is: 760760.339


In [347]:
#The price of the swap with a 5bps is  3.7953% + 0.0005

In [348]:
#question 2d)
#The market value of this swap 0
market_value = fixed_leg_pv - float_leg_pv
market_value

1.1641532182693481e-10

In [349]:
#Market value with 5bps fee:
market_value_fee = fixed_leg_pv_2c - float_leg_pv
market_value_fee

9606.298995143268

In [350]:
#Qustion 2e)
beg_date=date(2007,1,31)
last_date=date(2020,11,30)
delta=(last_date-beg_date).days

In [351]:
hist_dates=pd.date_range(beg_date,last_date,freq='d')
hist_dates

DatetimeIndex(['2007-01-31', '2007-02-01', '2007-02-02', '2007-02-03',
               '2007-02-04', '2007-02-05', '2007-02-06', '2007-02-07',
               '2007-02-08', '2007-02-09',
               ...
               '2020-11-21', '2020-11-22', '2020-11-23', '2020-11-24',
               '2020-11-25', '2020-11-26', '2020-11-27', '2020-11-28',
               '2020-11-29', '2020-11-30'],
              dtype='datetime64[ns]', length=5053, freq='D')

In [357]:
for b in hist_dates:
    try:
        swaps_hist=swaps.loc[str(b)].transpose()
        swaps_hist.columns=['par_rate']
        swaps_hist.reset_index(inplace=True)
        swaps_hist[['instrument', 'maturity']] = swaps_hist['index'].str.split(' ', expand=True) #splitting the index column to have a columns with only the name of the product
        swaps_hist["maturity"] = swaps_hist["maturity"].apply(lambda x: x + "M" if "X" in x else x) #adding a M to the FRAs maturities
        swaps_hist['maturity_fra'] = swaps_hist['maturity'].str.split('X').str[1] #getting the maturity of the fra
        swaps_hist.loc[swaps_hist["instrument"] == "FRA", "maturity"] = swaps_hist["maturity_fra"] #getting the new maturity
        swaps_hist["year_frac"] = swaps_hist["maturity"].apply(extract_number)
        swaps_hist.set_index('index',inplace=True)
        forward_hist=swaps_hist.loc['FRA 6X12', 'par_rate']
        one_year_rate_hist=(((1+swaps_hist.loc['Deposit 6M', 'par_rate'])**swaps_hist.loc['Deposit 6M', 'year_frac'])*(1+forward_hist)**(1-swaps_hist.loc['Deposit 6M', 'year_frac']))-1
        yield_curve_hist = swaps_hist[swaps_hist.index.str.contains("FRA") == False]
        one_year_rate_frame_hist = pd.DataFrame([[one_year_rate_hist,'-','1Y','-',1.0]],
            index=['Swap 1Y'],
            columns=['par_rate', 'instrument','maturity','maturity_fra','year_frac'])
        yield_curve_hist=yield_curve_hist.append(one_year_rate_frame_hist)
        yield_curve_hist.set_index('year_frac',inplace=True)
        yield_curve_hist.loc[0.5, 'df'] = 1 / (1 + (yield_curve_hist.loc[0.5, 'par_rate']/100) * 0.5)
        yield_curve_hist.loc[0.5, 'zero_rate'] = yield_curve_hist.loc[0.5, 'par_rate']/100
        yield_curve_hist.loc[1, 'df'] = 1 / (1 + yield_curve_hist.loc[1, 'par_rate']/100 * 1)
        yield_curve_hist.loc[1, 'zero_rate'] = yield_curve_hist.loc[1, 'par_rate']/100
        yield_curve_hist=yield_curve_hist.sort_index()
        yield_curve_hist["Tenor"]= yield_curve_hist.index
        yield_curve_hist['maturity'] = [date(b.year,b.month+6,b.day)] + [date(b.year + int(n), b.month, b.day) for n in yield_curve_hist.index[1:]]
        yield_curve_hist['days'] = yield_curve_hist.maturity.apply(lambda x: (x - b).days)
        idx_hist=0
        row_hist=0
        for idx_hist, row_hist in yield_curve_hist.loc[2:].iterrows():
            prev_dfs = yield_curve_hist.loc[1:idx_hist-1].df.sum()
            yield_curve_hist.loc[idx_hist, 'df'] = (1 - row_hist.par_rate/100 * prev_dfs) / (1+row_hist.par_rate/100)
        for a in yield_curve_hist.index[2:]:
            yield_curve_hist.loc[a,'zero_rate']=((1/yield_curve_hist.loc[a,'df'])**(1/a))-1
        
        
            
    except:
        continue
yield_curve_hist

,par_rate,instrument,maturity,maturity_fra,df,zero_rate,Tenor
year_frac,,,,,,,
0.50000,-0.50800,Deposit,6M,NaN,1.00255,-0.00508,0.50000
1.00000,-0.51101,-,1Y,-,1.00514,-0.00511,1.00000
2.00000,-0.51860,Swap,2Y,NaN,NaN,NaN,2.00000
3.00000,-0.50800,Swap,3Y,NaN,NaN,NaN,3.00000
4.00000,-0.48700,Swap,4Y,NaN,NaN,NaN,4.00000
5.00000,-0.45800,Swap,5Y,NaN,NaN,NaN,5.00000
6.00000,-0.42400,Swap,6Y,NaN,NaN,NaN,6.00000
7.00000,-0.38400,Swap,7Y,NaN,NaN,NaN,7.00000
8.00000,-0.34150,Swap,8Y,NaN,NaN,NaN,8.00000
